## ЛАБОРАТОРНАЯ РАБОТА ИТОГОВАЯ
## Статистическая обработка данных психологического исследования
#### по дисциплине «Математическая статистика»
Направление подготовки 01.03.02 Прикладная математика и информатика
Очной формы обучения

выполнил: студент группы Б9123-01.03.02ии

Иванов Матвей Олегович

принял: Деревягин Андрей Алексеевич

# Математическая часть

# листинг кода

## подготовка

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATA_PATH = "/home/matv864/it/AI_work/data/psydata.xlsx"
SIGNIFICANCE_LEVEL = 0.01

In [3]:
# заголовки убрал вручную
socio = pd.read_excel(DATA_PATH, sheet_name='Социофобия')
attach = pd.read_excel(DATA_PATH, sheet_name='ПривязанностьМ')
anxiety = pd.read_excel(DATA_PATH, sheet_name='Тревожность')
negative = pd.read_excel(DATA_PATH, sheet_name='Негатив')
fears = pd.read_excel(DATA_PATH, sheet_name='Страхи')

In [4]:
data = pd.DataFrame({'id': socio['id']})

data['социалочка'] = socio['Общий']

data['надёжная привязанность'] = attach['н']
data['тревожная привязанность'] = attach['т']
data['избегающая привязанность'] = attach['и']

data['общая тревожность'] = anxiety['Общая']
data['школьная тревожность'] = anxiety['Школьная']
data['самооценочная тревожность'] = anxiety['Самооценочная']
data['межличностная тревожность'] = anxiety['Межличностная']

data['страх негативной оценки'] = negative['Сумма']
data['сумма всех страхов из таблицы страхов'] = fears['Сумма']
data['общий бал страхов'] = data['страх негативной оценки'] + data['сумма всех страхов из таблицы страхов']





## тесты и инструменты

In [5]:
# проверка на нормальность через тесты Шапиро-Уилка и Колмогорова-Смирнова
def check_normality(*groups):
    results = []
    for group in groups:
        stat1, p1 = stats.shapiro(group)
        results.append(p1 > SIGNIFICANCE_LEVEL)

    if all(results):
        return True
    
    # тест Шапиро-Уилка чувствителен к выбросам, поэтому проверяем более универсальным тестом Колмогорова-Смирнова
    
    results = []
    for group in groups:
        ks_stat1, ks_p1 = stats.kstest(group, 'norm', args=(np.mean(group), np.std(group)))
        results.append(ks_p1 > SIGNIFICANCE_LEVEL)

    return all(results)

In [6]:
def compare_groups(column: str):
    """
    return
    normal: bool
    p_value (middle of unrel attach > middle of rel attach): float
    """
    unrel = data[data['надёжная привязанность'] > 5][column].dropna()  # X - среднее ненадёжной группы
    rel = data[data['надёжная привязанность'] <= 5][column].dropna()      # Y - среднее надёжной группы

    normal = check_normality(unrel, rel)
    stat, p = stats.ttest_ind(unrel, rel, alternative='greater')
    return normal, p


## задание 1,2,3,4

In [7]:
levels = [
    # 1 задание
    "социалочка",

    # 2 задание
    "общая тревожность",
    "школьная тревожность",
    "самооценочная тревожность",
    "межличностная тревожность",

    # 3 задание
    "страх негативной оценки",
    
    # 4 задание
    "общий бал страхов",
]

for level in levels:
    print(level)
    normal, p_val = compare_groups(level)

    if not normal:
        print("Нормальность не подтверждена, результаты t-теста стоит воспринимать осторожно")

    if p_val < 0.05:
        print(f"H_1 подтверждена — среднее значение уровня '{level}' ненадёжно привязанных выше, чем надёжно привязанных")
    else:
        print(f"H_0 не отвергается — нет доказательств, что среднее значение уровня '{level}' ненадёжно привязанных выше, чем надёжно привязанных")
    print("-"*100)


социалочка
H_0 не отвергается — нет доказательств, что среднее значение уровня 'социалочка' ненадёжно привязанных выше, чем надёжно привязанных
----------------------------------------------------------------------------------------------------
общая тревожность
H_0 не отвергается — нет доказательств, что среднее значение уровня 'общая тревожность' ненадёжно привязанных выше, чем надёжно привязанных
----------------------------------------------------------------------------------------------------
школьная тревожность
H_0 не отвергается — нет доказательств, что среднее значение уровня 'школьная тревожность' ненадёжно привязанных выше, чем надёжно привязанных
----------------------------------------------------------------------------------------------------
самооценочная тревожность
H_0 не отвергается — нет доказательств, что среднее значение уровня 'самооценочная тревожность' ненадёжно привязанных выше, чем надёжно привязанных
---------------------------------------------------------

## задание 4